In [1]:
import pandas as pd
import numpy as np
import numexpr as ne
from functools import reduce

In [23]:
df = pd.read_csv('GKX_20201231.csv')
f=['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 
             'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt', 'cashpr',  
             'cfp', 'cfp_ia', 'chatoia', 'chcsho', 
             'chempia', 'chinv', 'chmom',  'chpmia', 'chtx', 
             'cinvest', 'convind', 
             'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 
             'ear', 'egr', 'ep', 'gma', 'grcapx',  'grltnoa', 'herf', 
             'hire', 'idiovol', 'ill', 'indmom', 'invest',  'lev', 'lgr', 
             'maxret', 'mom12m', 'mom1m', 'mom36m', 'mom6m', 'ms', 'mvel1', 
             'mve_ia', 'nincr', 'operprof', 'orgcap', 
             'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchquick', 
             'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 
             'pchsaleinv', 'pctacc', 'pricedelay', 'ps', 'quick', 
             'rd', 'rd_mve', 'rd_sale', 'realestate', 
             'retvol', 'roaq', 'roavol', 
             'roeq', 'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 
             'secured', 'securedind', 'sgr', 'sin', 'sp',  
             'std_dolvol', 'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 
             'turn', 'zerotrade']
sec = ['permno', 'DATE', 'RET', 'prc', 'SHROUT','mve0','sic2']

In [24]:
def top_thresh_filter(group):
    prc = group['prc'].values.copy()
    if len(prc) > 1000:
        tred = np.sort(prc)[-1000]
    else:
        tred = 0
    return group[group['prc'] >= tred]
def bottom_thresh_filter(group):
    prc = group['prc'].values.copy()
    if len(prc) > 1000:
        tred = np.sort(prc)[999]
    else:
        tred = np.inf
    return group[group['prc'] <= tred]
top_df = df.groupby('DATE').apply(lambda x: top_thresh_filter(x))
bottom_df = df.groupby('DATE').apply(lambda x: bottom_thresh_filter(x))

In [32]:
df = bottom_df

In [33]:
macro = pd.read_excel('PredictorData2021.xlsx', parse_dates=['yyyymm'])
macro.yyyymm = macro.yyyymm.astype(int)
macro['dp'] = np.log(macro['D12']) - np.log(macro['Index'])
macro['ep'] = np.log(macro['E12']) - np.log(macro['Index'])
macro['bm'] = macro['b/m']
macro['tms'] = macro['lty'] - macro['tbl']
macro['dfy'] = macro['BAA'] - macro['AAA']
macro1 = macro.loc[(macro['yyyymm']>=195701)&(macro['yyyymm']<=201612),
                   ['yyyymm', 'dp', 'ep', 'bm', 'ntis', 'tbl', 'tms', 'dfy', 'svar']]
macro1.to_csv('macro.csv',index=False)
#macro = pd.read_csv('macro.csv')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [34]:
df = df.loc[(df['DATE']>=19570101)&(df['DATE']<=20161231)]

In [35]:
df['yyyymm'] = df['DATE'].astype(str).str[:6].astype(int)
data = pd.merge(df, macro, how='left', on='yyyymm', suffixes=('', '_macro'))

/var/folders/lk/c6gkc6x5345_rvndycqph7nm0000gn/T/ipykernel_85630/2012976093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['yyyymm'] = df['DATE'].astype(str).str[:6].astype(int)


In [38]:
data = data[['DATE','permno','RET']+f]

In [39]:
data.to_csv('bottom_data.csv',index=False)